**ALUNOS:**

CARLOS DINART SANTOS DE MORAIS JUNIOR,

DOUGLAS SILVA DE ALMEIDA,

LUCAS DO LAGO MONIZ DA SILVA

**TAMANHO DOS DATASETS:**

1- 2000 LINHAS 

2 - 891 LINHAS

3- 45222 LINHAS

##Dataset1 - Predição de casos de diabetes

Este dataset, com dados do hospital de Frankfurt, tem como objetivo tentar descobrir se a pessoa possui diabetes.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 10)
from sklearn.model_selection import train_test_split #Separar os dados
from sklearn.preprocessing import StandardScaler #preprocessar para evitar viés
from sklearn.neighbors import KNeighborsClassifier #
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Data Science/KNN/diabetes.csv', sep=',')
df

In [ ]:
#verificando se existem algum dado com 0
df.loc[df['BloodPressure']==0]

In [ ]:
#substituindo os valores iguais a zero pela média
sem_zero=['Glucose','BloodPressure','SkinThickness','Insulin','BMI']

for coluna in sem_zero:
  df[coluna] = df[coluna].replace(0, np.NaN)
  media = int(df[coluna].mean(skipna=True))
  df[coluna] = df[coluna].replace(np.NaN, media)

In [ ]:
#dataframe sem 0
df

In [ ]:
#dividindo o dataset em teste e treino
X = df.iloc[:, 0:8]
y = df.iloc[:, 8]
X_train, X_text, y_train, y_test = train_test_split(X,y, random_state=0, test_size = 0.2)

In [ ]:
#scaling 
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_text = sc_X.transform(X_text)

In [ ]:
#definindo um valor para K
int(len(y_train)**0.5)


In [ ]:
# criando o classificador
classifier = KNeighborsClassifier(n_neighbors=41, p=2, metric = 'euclidean')

In [ ]:
classifier.fit(X_train, y_train)

In [ ]:
y_pred = classifier.predict(X_text)
y_pred

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print(cm)
print("\n",f1_score(y_test, y_pred))

In [ ]:
antes = accuracy_score(y_test, y_pred)
antes

In [ ]:
print(classification_report(y_test, y_pred))

###Procurando o melhor valor para K


In [ ]:
from sklearn.model_selection import GridSearchCV
knn_params = {"n_neighbors": np.arange(1,20)}
knn = KNeighborsClassifier()
knn_cv = GridSearchCV(knn, knn_params, cv=10)
knn_cv.fit(X_train, y_train)

In [ ]:
print("melhor KNN score:" + str(knn_cv.best_score_))
print("Melhor valor para K: " + str(knn_cv.best_params_))

###Comparando a precisão para diferentes valores de K

In [ ]:
classifier = KNeighborsClassifier(n_neighbors=1, p=2, metric = 'euclidean')
classifier.fit(X_train, y_train)

In [ ]:
y_pred = classifier.predict(X_text)
depois = accuracy_score(y_test, y_pred)
depois

In [ ]:
d = {'Precisão com K = raiz quadrada do tamanho do teste': [antes], '/ Precisão com o GridSearchCV': [depois]}

In [ ]:
pd.DataFrame(data=d)

##Dataset 2 - TITANIC

Aqui será feita uma previsão se a pessoa sobreviveu ou morreu no Titanic

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Data Science/KNN/train.csv') # Salvamos com esse nome antes
df.head()

In [ ]:
df.shape

In [ ]:
df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Age'], inplace=True)
df.head()

In [ ]:
for col in df.columns:
    if df[col].dtype == object:
        print(col)

In [ ]:
df = pd.get_dummies(df)
df.head()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()
scaler.fit(df)
df = pd.DataFrame(scaler.transform(df), columns=df.columns)
print(df)

In [ ]:
df.head()

In [ ]:
target = 'Survived'
features = list(df.columns)
features.remove(target)

print('Features:', features)
print('Target:', target)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df[features], df[target], 
                                                    test_size=0.2, random_state=42)

print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)

In [ ]:
#descobrindo o valor de k
val_k = round(len(y_train)**0.5)
val_k

In [ ]:
# Importando nosso classificador
from sklearn.neighbors import KNeighborsClassifier

# Criando o modelo que usa k = sqrt(y_train)
clf = KNeighborsClassifier(n_neighbors=val_k)

# Treinando o modelo nos dados de treino
clf.fit(X_train, y_train)

In [ ]:
# Assim podemos usar o modelo para prever a classe de novos exemplos
y_pred = clf.predict(X_val)

In [ ]:
# Importar a função que calcula a acurácia
from sklearn.metrics import accuracy_score

acc = accuracy_score(y_val, y_pred)
print('Acurácia:', acc)

In [ ]:
# Criando lista com todos os números de vizinhos que queremos testar
n_vizinhos = np.arange(1, 11) # Array de 1 a 10

# Criando uma lista para guardarmos os resultados
resultados = []

# Para cada numero de vizinhos
for k in n_vizinhos:
    # Criamos um modelo novo e treinamos ele
    clf = KNeighborsClassifier(n_neighbors=k)
    clf.fit(X_train, y_train)
    
    # Fazendo predição para os dados de validação e calculando acurácia
    y_pred = clf.predict(X_val)
    acc = accuracy_score(y_val, y_pred)
    
    # Salvando a acurácia para o numero atual de vizinhos
    resultados.append(acc)

In [ ]:
# No eixo x teremos o número de vizinhos e no eixo y a acurácia
plt.plot(n_vizinhos, resultados, '-o')
# Definindo que no eixo x queremos exibir cada numero de vizinhos
plt.xticks(n_vizinhos);

In [ ]:
lista = [2,27]
resultados = []
for k in lista:
    clf = KNeighborsClassifier(n_neighbors=k)
    clf.fit(X_train, y_train)
    
    y_pred = clf.predict(X_val)
    acc = accuracy_score(y_val, y_pred)
    resultados.append(acc)


In [ ]:
# Comparando a Precisão para os 2 valores de K
plt.plot(lista, resultados, '-o')
plt.xticks(lista);

##DATASET 3 - SALÁRIO


Neste dataset nós vamos tentar prever se a pessoa tem um salário maior ou menor que 50 mil dolares

In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt


## **Introdução**

In [ ]:
#importando o dataset
df = pd.read_csv('/content/drive/MyDrive/Data Science/KNN/adult.csv', sep=',')

In [ ]:
df.shape

## **Limpando os Dados**

Apagando as celulas com valores desconhecidos

In [ ]:
df = df.replace('?',np.NaN)
df = df.dropna(axis = 0)
df = df.reset_index(drop=True)
df

Checando os valores que estão contidos na coluna income

In [ ]:
df['income'].value_counts()

Convertendo o salario para 0 e 1, sendo '1' equivalente a menor ou igual a 50k e '0' maior que 50k por ano

In [ ]:
df['income'] = (df['income']==("<=50K")).astype(int)

df['income']

## **Dividindo o DataFrame em int e str**

In [ ]:
df_str = df.select_dtypes(include='object')
df_int = df.select_dtypes(exclude='object')
df_str.describe(include='all')


In [ ]:
df_str.head()

In [ ]:
df_int.head()

In [ ]:
df_str = pd.get_dummies(df_str)
df_str.head()

###Visualizando os dados através de graficos

In [ ]:
import seaborn as sns

In [ ]:
df_cat = df.select_dtypes(include = 'object')



In [ ]:
for i in ['workclass', 'education', 'marital-status', 'occupation','relationship', 'race', 'gender', 'native-country']:
  sns.countplot(x=i, data = df_cat)
  plt.xticks(rotation=90)
  plt.show()

## **Normalizing numeric part**

In [ ]:
y_last = df_int['income']
x = df_int.drop(columns='income')
column_names = x.columns.values
type(x)

In [ ]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()
x_stndrd = scaler.fit_transform(x)
type(x_stndrd)

Change x_stndrd's type, because we cannot combine str and int data.

In [ ]:
x_stndrd = pd.DataFrame(x_stndrd)
x_stndrd.columns = column_names
type(x_stndrd)

In [ ]:
x_last = pd.concat([x_stndrd,df_str],axis=1)
x_last

In [ ]:
x_last

## **Train Test Split**

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test  = train_test_split(x_last,y_last,test_size=0.05,random_state=42)

## **Aplicando o KNN**

In [ ]:
#definindo k como raiz quadrada do tamanho do conjunto de treinamento  213
n_neighbors = int(len(y_train)**0.5)
n_neighbors

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors).fit(x_train,y_train)

In [ ]:
y_hat = neigh.predict(x_test)

In [ ]:
from sklearn import metrics
metrics.accuracy_score(y_test,y_hat)

## selecionando o melhor valor para 'K'

In [ ]:
Ks=15
mean_acc = np.zeros((Ks-1))
std_acc = np.zeros((Ks-1))

for n in range(1,Ks):
    neigh = KNeighborsClassifier(n_neighbors = n).fit(x_train,y_train)
    y_hat = neigh.predict(x_test)
    mean_acc[n-1] = metrics.accuracy_score(y_test,y_hat)
    std_acc[n-1] = np.std(y_hat==y_test)/np.sqrt(y_hat.shape[0])
    
mean_acc

In [ ]:
plt.plot(range(1,Ks),mean_acc,'g',marker='o')
plt.fill_between(range(1,Ks),mean_acc - 1 * std_acc,mean_acc + 1 * std_acc, alpha=0.10)
plt.legend(('Accuracy ', '+/- 3xstd'))
plt.ylabel('Accuracy ')
plt.xlabel('Number of Neighbors (K)')
plt.tight_layout()
plt.show()

Assumindo K como 13

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=13).fit(x_train,y_train)
y_hat = neigh.predict(x_test)
metrics.accuracy_score(y_test,y_hat)

# **Confusion Matrix**

In [ ]:
from sklearn.metrics import plot_confusion_matrix, confusion_matrix
confusion_matrix(y_test, y_hat)

In [ ]:
plot_confusion_matrix(neigh,x_test,y_test)